# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import datetime

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")


In [5]:
filepath = '/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json'

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df.loc[:,["song_id","title","artist_id","year","duration"]].values[0].tolist()
song_data

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df.loc[:,["artist_id","artist_name","artist_location","artist_latitude","artist_longitude"]].values[0].tolist()
artist_data

['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files("data/log_data")

In [12]:
filepath = "/home/workspace/data/log_data/2018/11/2018-11-29-events.json"

In [13]:
df = pd.read_json(filepath, lines=True)
df.tail()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
361,None,Logged In,Tegan,F,37,Levine,NaN,paid,"Portland-South Portland, ME",GET,Home,1.540794e+12,1065,None,200,1543532195796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
362,Pearl Jam,Logged In,Tegan,F,38,Levine,283.89832,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Rearviewmirror,200,1543532242796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
363,The 2 Live Crew,Logged In,Tegan,F,39,Levine,188.31628,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Me So Horny,200,1543532525796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
364,311,Logged In,Tegan,F,40,Levine,113.26649,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Loco,200,1543532713796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
365,Light Of The World,Logged In,Kynnedi,F,0,Sanchez,207.07220,free,"Cedar Rapids, IA",PUT,NextSong,1.541079e+12,1029,"Midnight Groovin' (7"" Version) (2006 Digital R...",200,1543533515796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5...",89


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df.page=="NextSong"]
df.tail(10)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
355,The White Stripes,Logged In,Tegan,F,32,Levine,194.48118,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Bone Broke,200,1543531178796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
356,Furthermore,Logged In,Tegan,F,33,Levine,211.69587,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Are You The Walrus?,200,1543531372796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
357,Eminem / Bizarre,Logged In,Tegan,F,34,Levine,244.55791,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Amityville,200,1543531583796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
358,Florence + The Machine,Logged In,Tegan,F,35,Levine,185.31220,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,I'm Not Calling You A Liar,200,1543531827796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
359,Soda Stereo,Logged In,Braden,M,0,Parker,316.73424,free,"Youngstown-Warren-Boardman, OH-PA",PUT,NextSong,1.540999e+12,1007,Signos,200,1543531878796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",74
360,Taylor Swift,Logged In,Tegan,F,36,Levine,230.47791,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,You Belong With Me,200,1543532012796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
362,Pearl Jam,Logged In,Tegan,F,38,Levine,283.89832,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Rearviewmirror,200,1543532242796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
363,The 2 Live Crew,Logged In,Tegan,F,39,Levine,188.31628,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Me So Horny,200,1543532525796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
364,311,Logged In,Tegan,F,40,Levine,113.26649,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Loco,200,1543532713796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
365,Light Of The World,Logged In,Kynnedi,F,0,Sanchez,207.07220,free,"Cedar Rapids, IA",PUT,NextSong,1.541079e+12,1029,"Midnight Groovin' (7"" Version) (2006 Digital R...",200,1543533515796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5...",89


In [15]:
df.info()
t = pd.to_datetime(df['ts'], unit='ms', origin='unix')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 319 entries, 0 to 365
Data columns (total 18 columns):
artist           319 non-null object
auth             319 non-null object
firstName        319 non-null object
gender           319 non-null object
itemInSession    319 non-null int64
lastName         319 non-null object
length           319 non-null float64
level            319 non-null object
location         319 non-null object
method           319 non-null object
page             319 non-null object
registration     319 non-null float64
sessionId        319 non-null int64
song             319 non-null object
status           319 non-null int64
ts               319 non-null int64
userAgent        319 non-null object
userId           319 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 47.4+ KB


In [16]:
time_data =  df.ts
hour = t.dt.hour
day = t.dt.day
week = t.dt.week
month = t.dt.month
year = t.dt.year
weekday = t.dt.weekday
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']


In [17]:
time_df = pd.DataFrame(data=list(zip(time_data, hour, day, week, month, year, weekday)), columns = column_labels)
time_df.tail()

,start_time,hour,day,week,month,year,weekday
314,1543532012796,22,29,48,11,2018,3
315,1543532242796,22,29,48,11,2018,3
316,1543532525796,23,29,48,11,2018,3
317,1543532713796,23,29,48,11,2018,3
318,1543533515796,23,29,48,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df.loc[:, ["userId","firstName", "lastName", "gender", "level"]]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
        print(songid, artistid)
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [23]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *


def process_song_file(cur, filepath):
    # open song file
    df = pd.read_json(filepath, lines=True)

    # insert song record
    song_data = df.loc[:,["song_id","title","artist_id","year","duration"]].values[0].tolist()
    cur.execute(song_table_insert, song_data)
    
    # insert artist record
    artist_data = df.loc[:,["artist_id","artist_name","artist_location","artist_latitude","artist_longitude"]].values[0].tolist()
    cur.execute(artist_table_insert, artist_data)


def process_log_file(cur, filepath):
    # open log file
    df = pd.read_json(filepath, lines=True)

    # filter by NextSong action
    df = df[df.page=="NextSong"]

    # convert timestamp column to datetime
    t = pd.to_datetime(df['ts'], unit='ms', origin='unix')
    
    # insert time data records
    time_data = df.ts
    column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']
    time_df = pd.DataFrame(data=list(zip(time_data, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday)), columns = column_labels)

    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))

    # load user table
    user_df = df.loc[:, ["userId","firstName", "lastName", "gender", "level"]]

    # insert user records
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)

    # insert songplay records
    for index, row in df.iterrows():
        
        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # insert songplay record
        songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)


def process_data(cur, conn, filepath, func):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed.'.format(i, num_files))


def main():
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()

    process_data(cur, conn, filepath='data/song_data', func=process_song_file)
    process_data(cur, conn, filepath='data/log_data', func=process_log_file)

    conn.close()


if __name__ == "__main__":
    main()

85 files found in data/song_data
1/85 files processed.
2/85 files processed.
3/85 files processed.
4/85 files processed.
5/85 files processed.
6/85 files processed.
7/85 files processed.
8/85 files processed.
9/85 files processed.
10/85 files processed.
11/85 files processed.
12/85 files processed.
13/85 files processed.
14/85 files processed.
15/85 files processed.
16/85 files processed.
17/85 files processed.
18/85 files processed.
19/85 files processed.
20/85 files processed.
21/85 files processed.
22/85 files processed.
23/85 files processed.
24/85 files processed.
25/85 files processed.
26/85 files processed.
27/85 files processed.
28/85 files processed.
29/85 files processed.
30/85 files processed.
31/85 files processed.
32/85 files processed.
33/85 files processed.
34/85 files processed.
35/85 files processed.
36/85 files processed.
37/85 files processed.
38/85 files processed.
39/85 files processed.
40/85 files processed.
41/85 files processed.
42/85 files processed.
43/85 file